# Call Libraries

# Import Common Functions

In [ ]:
%run Common.ipynb

# Input Parameter

In [ ]:
Horizon = 8
Stock_Code = 'VOO'
IsDown = False

In [ ]:
csv_path = f"{RNN_PATH}/{Stock_Code}/Analysis/Model_Result.csv"

## Get Best variables combination for specific Horizon (Notebook : main_RNN_Analysis)

In [ ]:
def Fn_GetBest( target_horizon ):
    
    df_result = pd.read_csv( csv_path)
    
    numbers = df_result['Horizon'].drop_duplicates().tolist()

    closest_horizon = min(numbers, key=lambda x: abs(x - target_horizon))

    best_row = tuple(df_result[df_result['Horizon'] == closest_horizon].sort_values('mae_aggr').head(1).iloc[0])[:6]

    id, horizon ,row_cnt , layer , train_fraction, seq_len = best_row

    return int(id), int(row_cnt), int(layer), float(train_fraction), int(seq_len)

## For model training

In [ ]:
id, d, l, t, p = Fn_GetBest( Horizon )

## For Download Data

In [ ]:
Data_Year = 20
end = datetime.now()
NextMonth = 1 if end.month == 12 else end.month + 1
start = datetime.now().replace(year = end.year - Data_Year).replace(day=1).replace(month = NextMonth)

# Get Data

In [ ]:
if IsDown:

    Download_Stock(Stock_Code, start, end , f"{SOURCE_PATH}/{Stock_Code}.csv" )


path = f'{SOURCE_PATH}/{Stock_Code}.csv'


df = pd.read_csv(path)[['Code','Date','Close']]
df['Date'] = pd.to_datetime(df['Date'])

print(f"There are {len(df)} days in total ")

d = d if d < len(df) else len(df)

df = df.sort_values(by='Date', ascending=True).reset_index(drop=True)

Input_df = df[-d:].copy()

"""
scaler = MinMaxScaler((0,1))
scaler.fit(Input_df[['Close']])
Input_df['Scaled_Close'] = scaler.transform(Input_df[['Close']])
"""

print(f"Got {len(Input_df)} of days for model training")



In [ ]:
Plot_Result(df, 'Current Data',Stock_Code )
Plot_Result(Input_df, 'Current Data',Stock_Code )

# Train RNN model

In [ ]:
%run RNN_Train_Horizon.ipynb

In [ ]:
mae, mae_model, model, data_df, indi_df = RNN(Input_df, p , t , Horizon, id , d, True)